# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [12]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [31]:
document_tree = ET.parse( './data/mondial_database_less.xml' )
child=document_tree.find('name')
child

In [16]:
# print names of all countries
for child in document_tree.getroot():
    print (child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [23]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('* ' + element.find('name').text + ':')
    #print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [67]:
%matplotlib inline
import pandas as pd
import numpy as np

In [219]:
document = ET.parse( './data/mondial_database.xml' )

In [367]:
df=pd.DataFrame(columns=('country', 'infant_mortality'))
for element in document.iterfind('country'):
    infant_mortality=element.findtext('infant_mortality')
    country=element.findtext('name')
    row = dict(zip(['country', 'infant_mortality'], [country, infant_mortality]))
    row_s=pd.Series(row)
    df= df.append(row_s,ignore_index=True)
    #df[1]= df.append([infant_mortality])
    #df = df.append([infant_mortality],ignore_index='True')
    #df = df.append(row2,ignore_index='True')
    #print ('* ' + element.find('name').text + ':' + element.find('infant_mortality').text)
    

In [368]:
df['infant_mortality']=df.infant_mortality.astype(float)


In [369]:
df.sort_values(by='infant_mortality').head(10)

,country,infant_mortality
38,Monaco,1.81
98,Japan,2.13
117,Bermuda,2.48
36,Norway,2.48
106,Singapore,2.53
37,Sweden,2.60
10,Czech Republic,2.63
78,Hong Kong,2.73
79,Macao,3.13
44,Iceland,3.15


In [503]:
df_city=pd.DataFrame(columns=('country', 'city','lat','lon','population'))
for element in document.iterfind('country'):
    #using xpath to find city (select all grand child with city)
    for subelement in element.iterfind('*/city'):
        country=element.findtext('name')
        city = subelement.findtext('name')
        lat = subelement.findtext('latitude')
        lon = subelement.findtext('longitude')
        #for subelement1 in element.iterfind('city'):
        #using xpath to find last element in population 
        population = subelement.findtext('population[last()]')
        population=population
        row = dict(zip(['country','city','lat','lon','population'], [country,city,lat,lon,population]))
        row_s=pd.Series(row)
        df_city= df_city.append(row_s,ignore_index=True)
    

In [504]:
df_city[df_city.city=='Chennai']

,country,city,lat,lon,population
1484,India,Chennai,13.0667,80.25,4646732


In [505]:
df_city['population']=df_city.population.astype(float)
df_city.sort_values(by='population',ascending=False).head(10)

,country,city,lat,lon,population
1278,China,Shanghai,31.2,121.433,22315474.0
738,Turkey,Istanbul,41.0,28.95,13710512.0
1455,India,Mumbai,18.9,72.8167,12442373.0
458,Russia,Moskva,55.7667,37.6667,11979529.0
1277,China,Beijing,39.5,116.2,11716620.0
2490,Brazil,São Paulo,-23.55,-46.6333,11152344.0
1279,China,Tianjin,39.1,117.2,11090314.0
1001,China,Guangzhou,23,114,11071424.0
1510,India,Delhi,28.5833,77.2,11034555.0
1004,China,Shenzhen,22.55,114.1,10358381.0


In [499]:
#10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
df_ethnic=pd.DataFrame(columns=('country', 'ethnic','ethnic_population'))
for element in document.iterfind('country'):
    #using xpath to find city (select all grand child with city)
    for ethnicgroup in element.iterfind('ethnicgroup'):
        country=element.findtext('name')
        ethnic_per = str(ethnicgroup.attrib)#element.findtext('.//ethnicgroup')
        ethnic = ethnicgroup.text
        population = element.findtext('population[last()]')
        population=population
        row = dict(zip(['country','ethnic','ethnic_population'], [country,ethnic,round((float(ethnic_per[16:-2])*int(population))/100)]))
        row_s=pd.Series(row)
        df_ethnic= df_ethnic.append(row_s,ignore_index=True)

In [500]:
df_ethnic[df_ethnic.country =='India'].head()
#ethnic_per[-5:-2]
#ethnic_per=str(ethnic_per)

,country,ethnic,ethnic_population
220,India,Dravidian,302713744.0
221,India,Indo-Aryan,871815583.0
222,India,Mongol,36325649.0


In [506]:
df_ethnic.sort_values(by='ethnic_population',ascending=False).head(10)

,country,ethnic,ethnic_population
176,China,Han Chinese,1.245059e+09
221,India,Indo-Aryan,8.718156e+08
220,India,Dravidian,3.027137e+08
345,United States,European,2.549581e+08
520,Nigeria,African,1.626516e+08
212,Bangladesh,Bengali,1.467769e+08
299,Japan,Japanese,1.265342e+08
93,Russia,Russian,1.146462e+08
278,Indonesia,Javanese,1.134560e+08
461,Brazil,European,1.088867e+08


In [ ]:
country_codes = {}
for country in document.findall('country'):
    country_codes[country.attrib['car_code']] = country.find('name').text  

In [625]:
#name and country of a) longest river, b) largest lake and c) airport at highest elevation
df_river=pd.DataFrame(columns=('country', 'river','length'))
for r in document.iterfind('river'):
    river=r.findtext('name')
    length=r.findtext('length')
    #country_s = str(r.attrib)#element.findtext('.//ethnicgroup')
    #country =country_s.rfind("country")
    country=country_codes[r.attrib['country'].split(' ')[0]]
    row = dict(zip(['country','river','length'], [country,river,length]))
    row_s=pd.Series(row)
    #row_s.country=row_s.country.find("country", start=0)
    df_river= df_river.append(row_s,ignore_index=True)

In [626]:
df_river['length']=df_river.length.astype(float)
df_river.sort_values(by='length',ascending=False).head(1)

,country,river,length
174,Colombia,Amazonas,6448.0


In [636]:
df_lake=pd.DataFrame(columns=('country', 'lake','area'))
for r in document.iterfind('lake'):
    lake=r.findtext('name')
    area=r.findtext('area')
    #country_s = str(r.attrib)#element.findtext('.//ethnicgroup')
    #country =country_s.rfind("country")
    country=country_codes[r.attrib['country'].split(' ')[0]]
    row = dict(zip(['country','lake','area'], [country,lake,area]))
    row_s=pd.Series(row)
    #row_s.country=row_s.country.find("country", start=0)
    df_lake= df_lake.append(row_s,ignore_index=True)

In [630]:
df_lake['area']=df_lake.area.astype(float)
df_lake.sort_values(by='area',ascending=False).head(1)

,country,lake,area
54,Russia,Caspian Sea,386400.0


In [683]:
df_airport=pd.DataFrame(columns=('country', 'airport','elevation'))
for r in document.iterfind('airport'):
    airport=r.findtext('name')
    elevation=r.findtext('elevation')
    #country_s = str(r.attrib)#element.findtext('.//ethnicgroup')
    #country =country_s.rfind("country")
    country=country_codes[r.attrib['country'].split(' ')[0]]
    row = dict(zip(['country','airport','elevation'], [country,airport,elevation]))
    row_s=pd.Series(row)
    #row_s.country=row_s.country.find("country", start=0)
    df_airport= df_airport.append(row_s,ignore_index=True)

In [698]:
df_airport['elevation'][df_airport.elevation=='']='0'
df_airport['elevation']=df_airport.elevation.astype(float)
df_airport.sort_values(by='elevation',ascending=False).head(1)

,country,airport,elevation
80,Bolivia,El Alto Intl,4063.0
